## Crypto Currency and Portfolio Construction

- Ryan Milgrim and Roo Fernando
- University of Washington
- CFRM 521: Machine Learning
- June, 3, 2024

### 1. Project Design [20 Points]

- Clear and Percise Problem Statement
- Reasonable Project Goal
- Extra credit of up to 10 points will be given for innovative content

#### Problem Statement

Cryptocurrency markets, first introduced in 2009, have seen widespread adoption, with two countries now recognizing cryptocurrencies as legal tender. Despite their speculative nature, the popularity of these digital assets continues to grow globally. Traditional investment strategies are typically grounded in economic fundamentals such as company earnings and market interest rates. However, applying these principles to cryptocurrencies is challenging due to their decentralized nature, lack of ties to any single entity, and extreme price volatility. This has made constructing and managing cryptocurrency portfolios particularly difficult for traditional investors.

In this project, we aim to develop and demonstrate the application of machine learning methods to construct cryptocurrency strategies that improve returns and reduce the risks associated with these investments. By leveraging these advanced techniques, we seek to provide investors with tools to navigate the highly volatile cryptocurrency markets more effectively, thereby enhancing their ability to make informed investment decisions.

#### Project Goal

The goal of our project is to develop a machine learning model capable of dynamically adjusting a cryptocurrency portfolio using historical prices and additional derived features. This model will either forecast expected prices or returns for our selected cryptocurrencies or suggest optimal allocations among these currencies. To evaluate the utility of these methods, we will implement a systematic investment process based on the model’s outputs and compare its performance against naive portfolio construction methods, such as a buy-and-hold strategy with perfect foresight of future returns (a capability our methods will not assume). If our model can achieve out-of-sample returns that exceed those of any individual cryptocurrency, it would demonstrate that our approach enables investors to make more informed and effective decisions in the cryptocurrency domain.

### 2. Data Processing and Feature Engineering  [25 Points]

#### Reliable and relevant data sources

The data we will be using for our analysis can be found here: <a href="https://www.kaggle.com/code/adityamhaske/crypto-currencies-price-analysis">Kaggle - Crypto Currencies Price Analysis</a>. We have consolidated the four CSV files into a single data.csv file to streamline our notebook. 

The dataset includes historical open, high, low, and closing prices for BTC, ETH, LTC, and XRP from January 1, 2018. Our analysis primarily relies on closing prices to calculate daily returns and other features.

It's important to note that our dataset contains some missing values due to exchange issues. Instead of discarding these rows, we have opted to fill missing prices with the last known price, as this method minimizes data loss and maintains the integrity of our time series.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the datafile and format as a time series.
data = pd.read_csv('data.csv', index_col=1)
data.index = pd.to_datetime(data.index)#, format='mixed')

# Format the dataset into features denoted as {Coin}__{Feature}
data = data.set_index(['Crypto'], append=True).unstack()
data.columns = [f'{coin}__{ohlc}' for ohlc, coin in data.columns.values]

# Sorting the timeseries by date and imputing missing values. 
data.sort_index()
data = data.ffill()
data.head()

#### Reasonable data cleaning and preprocessing

For our analysis, we create an additional feature named "Return" representing the day of day change in closign price. To ensure that our dataset is formatted well and free of errors, we visualize the dataset with the below code chunk. 

The code generates a histogram of daily returns and a time series plot of closing prices. Upon reviewing these visualizations, we noticed an unusual pattern in the histogram for XRP. Further inspection revealed that the dataset contains only two decimal places of precision for XRP, which often trades below $1.00. This lack of precision results in XRP frequently showing a 0% daily return.

To address this issue, we have decided to exclude XRP from our analysis and focus on the remaining three cryptocurrencies: BTC, ETH, and LTC

In [ ]:
def plot_returns_and_prices(coin, returns, prices):
    """Simple Plot Function to visualize the quality of our data"""

    # Making the plot
    fig, ax = plt.subplots(2, 1)
    returns.hist(bins=100, ax=ax[0])
    prices.plot(ax=ax[1])

    # Apply titles
    ax[0].set_title('Daily Returns')
    ax[1].set_title('Closing Prices')

    # Apply Grids for both plots
    ax[0].grid(True, alpha=0.7, linestyle='--')
    ax[1].grid(True, alpha=0.7, linestyle='--')

    # Apply title and layout
    fig.suptitle(coin)
    plt.tight_layout()

In [ ]:
# Plotting the returns and prices for each coin
coins = ['BTC', 'ETH', 'LTC', 'XRP']
for coin in coins:
    coin_returns = f'{coin}__Returns'
    coin_prices = f'{coin}__Close'

    # Appending the new Returns Feature
    data[coin_returns] = data[coin_prices].pct_change()
    plot_returns_and_prices(coin, data[coin_returns], data[coin_prices])

# Droping XRP due to our concerns about data quality.
coins = ['BTC', 'ETH', 'LTC']
data = data.drop(columns=[
    'XRP__Open',
    'XRP__High',
    'XRP__Low',
    'XRP__Close',
    'XRP__Returns',
])

#### Effective feature selection and generation

Predicting expectations of the future based on historical open / high / low / close / returns is quite difficult, some would argue impossible without incorpoating additional exogenous data. We did consider incorporating economic variables such as GDP and CPI, however, we have choosen to use technical indicators derived from historical prices instead. The indcators which we utilize are described below:

- **Volatility**: Also known as standard deviation, scaled by the window size of *n* days.

- **Momentum**: This indicator was developed through our own creativity and its calculation can be seen below. This indicator aims to capture how above or below the closing price is relative to the moving average and volatility.

- **RSI**: RSI measures the speed and magnitude of a security's recent price changes, it is used by technical traders to assess weather and asset is overbought, or oversold. Since crypto-currencies don't have any fundamentals, we believe including a very common technical indicator would be useful. 

<p>Momentum = <sup>(Close - Moving Average)</sup> / <sub>(Moving Average * Volatility)</sub></p>

<p>RSI = 100 - <sup> (100)</sup> / <sub>(1 + (Average Gain/ Average loss))</sub></p>


We suspect these technical indicators will provide insight into future prices, but this does come with a small cost in terms of data descruction. We considered using a custom transformer class to address the issue of missing data, however our attempts of this led to poor implemtations when it comes to splitting data into training / validation / testing sets. As a result, we will simply drop the first 40 rows of data which contains missing values.

In [ ]:
def calculate_rsi(prices, window=40):
    # Calculate the price differences
    delta = prices.diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the average gain and loss
    avg_gain = gain.rolling(window=window, min_periods=40).mean()
    avg_loss = loss.rolling(window=window, min_periods=40).mean()

    # Calculate the relative strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

def transform_prices(prices, window=40):
    """Function to transform prices into volatility and momentum indicators."""
    features = pd.DataFrame(index=prices.index)

    # Creating the new features
    features['Returns'] = prices.pct_change()    
    features['MovingAverage'] = prices.rolling(window=window).mean()
    features['Volatility'] = features['Returns'].rolling(window=window).std() * np.sqrt(window)
    features['RSI'] = calculate_rsi(prices)

    # Creating the momentum feature
    features['Momentum'] = prices - features['MovingAverage']
    features['Momentum'] /= features['MovingAverage'] * features['Volatility']

    # Droping features which the function should not append
    features = features.drop(columns=['MovingAverage', 'Returns'])

    # Returning the new features
    return features

# Using the function to append new features to the data
for coin in coins:
    features = transform_prices(data[coin + '__Close'])
    data = pd.concat([data, features.add_prefix(f'{coin}__')], axis=1)

data = data.sort_index(axis=1)
data = data.dropna()
data.info()

The below code chunk provides a visual of our indicators against the log of the closing price. We show these charts to provide the reader with insight into how our technical indcators behave overtime. 

In [ ]:
# Plot the technical indicators for each coin against its closing price.
figsize = (10, 7)
for coin in coins:

    # Plot the technical indicators.
    fig, ax = plt.subplots(4, 1, figsize=figsize)
    fig.suptitle(coin)

    # Plot the log of price, volatility, and momentum.
    np.log(data[f'{coin}__Close']).plot(ax=ax[0], title='Log of Closing Price')
    data[f'{coin}__Volatility'].plot(ax=ax[1], title='Volatility')
    data[f'{coin}__Momentum'].plot(ax=ax[2], title='Momentum')
    data[f'{coin}__RSI'].plot(ax=ax[3], title='RSI')

    # Apply Grids for all axs
    for axi in ax:
        axi.grid(True, alpha=0.7, linestyle='--')

    plt.tight_layout()
    plt.show()

### 3. Model Selection and Implementation [30 Points]

- Good reasons for choosing models
- Reasonable train/test/validation split
- Clear model training and tuning process

#### Plotting and Analysis Functions

For our analysis / results, we have written the below section of functions to abstract away much of the mundane / reusable code to make our project more readable. Please feel free to skip this section if you wish. 

In [ ]:
# Creating a returns dataframe for analysis
returns = pd.DataFrame()
for coin in coins:
    returns[coin] = data[coin + '__Returns']

In [ ]:
def analyze_regimes_and_get_best_portfolios(returns, regimes):
    """Function to print the average returns of each coin by regime and return portfolio dictionaries."""

    n_regimes = len(np.unique(regimes))
    regime_returns = returns.loc[regimes.index]

    # Print the average returns of each coin by regime and make a portfolio for each regime
    portfolios_by_regime = {}
    for i in range(n_regimes):
        average_returns = regime_returns[regimes == i].mean()

        print(f'\n-- {average_returns.idxmax()} Best coin in Regime {i}')
        print(average_returns)

        # Add the best portfolio for this regime to the dictionary
        best_coin_index = average_returns.idxmax()
        portfolios_by_regime[i] = [1 if coin == best_coin_index else 0 for coin in average_returns.index]

    # Also print out the best coin overall
    overall_average_returns = returns.mean()
    print(f'\n-- Best Coin Overall: {overall_average_returns.idxmax()}')

    # Create the best overall portfolio
    best_coin_index = overall_average_returns.idxmax()
    best_overall_portfolio = [1 if coin == best_coin_index else 0 for coin in overall_average_returns.index]
    best_overall_portfolio = {i: best_overall_portfolio for i in range(n_regimes)}

    return portfolios_by_regime, best_overall_portfolio

In [ ]:
def create_portfolio_timeseries(regimes, portfolio_dict, coins):
    """Function to create a portfolio timeseries from a regime timeseries and portfolio dictionary"""

    # Create a portfolio timeseries
    portfolio = pd.DataFrame(np.zeros((len(regimes), len(coins))), index=regimes.index, columns=coins)
    for regime, allocation in portfolio_dict.items():
        portfolio[regimes == regime] = allocation

    return portfolio

In [ ]:
def plot_portfolio_backtest(portfolio, benchmark, returns):
    """Function to plot the cumulative returns of a portfolio and its allocation over time"""

    # Calculate the cumulative returns of the portfolio and benchmark
    portfolio_returns = np.sum(returns * portfolio, axis=1)
    benchmark_returns = np.sum(returns * benchmark, axis=1)

    # Create the figure and axis
    fig, ax = plt.subplots(2, 1, figsize=(8, 5))

    # Plot the cumulative returns of the portfolio and benchmark
    ax[0].set_title('Cumulative Returns')
    (1 + portfolio_returns).cumprod().plot(ax=ax[0], label='Portfolio')
    (1 + benchmark_returns).cumprod().plot(ax=ax[0], label='Benchmark', linestyle='--')
    ax[0].grid(True, alpha=0.7, linestyle='--'), ax[0].legend()

    # Plot the rolling 30-day volatility of the portfolio and benchmark
    ax[1].set_title('Rolling 30-Day Volatility')
    portfolio_returns.rolling(window=30).std().plot(ax=ax[1], label='Portfolio')
    benchmark_returns.rolling(window=30).std().plot(ax=ax[1], label='Benchmark', linestyle='--')
    ax[1].grid(True, alpha=0.7, linestyle='--'), ax[1].legend()

    plt.tight_layout()
    plt.show()

    # Print out simple Sharpe ratio metrics
    portfolio_sharpe = portfolio_returns.mean() / portfolio_returns.std()
    benchmark_sharpe = benchmark_returns.mean() / benchmark_returns.std()
    print('\nSharpe Ratio:')
    print(f'Model: {portfolio_sharpe:.4f}')
    print(f'Benchmark: {benchmark_sharpe:.4f}')

    # Create a plot function which plots the portfolio allocations over time
    if portfolio_sharpe > benchmark_sharpe:
        print('Model Outperformed the benchmark.')
    else:
        print('Model Failed to beat the benchmark.')

In [ ]:
def plot_portfolio_allocations(portfolio, benchmark):

    # Create the figure and axis
    fig, ax = plt.subplots(2, 1, figsize=(8, 5))
    fig.suptitle('Model and Benchmark Allocations')

    # Plot the portfolio allocation
    portfolio.plot(ax=ax[0], kind='area', stacked=True)   
    ax[0].set_ylim((0, 1)), ax[0].set_ylabel('Portfolio'), ax[0].legend()
    ax[0].grid(True, alpha=0.7, linestyle='--')

    # Plot the portfolio allocation without x or y axis labels
    benchmark.plot(ax=ax[1], kind='area', stacked=True)
    ax[1].set_ylim((0, 1)), ax[1].set_ylabel('Benchmark'), ax[1].legend()
    ax[1].grid(True, alpha=0.7, linestyle='--')

    plt.tight_layout()
    plt.show()

#### KMeans Persistent Regime Model

For our first model, we will keep things simple and assume that our regimes persist for a single day. This will inform our prediction of which regime will occur on day t+1, and we will also assume that the optimal coin to hold during each regime will remain the same between our training set and our testing set. With these simple assumptions, we can begin to build a portfolio strategy. 

We will use 60% of the data for training and 40% of the data for testing. Because we have no hyper parameters to tune, we will not concern ourselves with hyper parameter tuning for now. Time series data should not be shuffeled, and the benchmark we hope to beat, is the best returning coin's performance of the testing dataset.

Here we introduce a simple pipeline which applies a robust scaler to our indicators then fits a kmeans model for n_clusters from 2 through 10 to determine the optimal number of clusters. After our analysis, we found that 2 clusters were optimal. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans

# Create the pipeline to only momentum and volatility columns
pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('kmeans', KMeans(n_init=10, random_state=42))
])

# Create an X dataset with only the technical indicators for regime detection
technical_indicators = [coin + '__Momentum' for coin in coins]
technical_indicators += [coin + '__Volatility' for coin in coins]
X = data[technical_indicators]


In [ ]:
from sklearn.metrics import silhouette_score

# Fit the pipeline to the for n_clusters = {2 - 10} and record silhouette score.
scores = dict()
for n in range(2, 11):
    pipeline.set_params(kmeans__n_clusters=n)
    pipeline.fit(X)
    
    # Transform data to get cluster labels
    labels = pipeline['kmeans'].labels_

    # Calculate the silhouette score
    score = silhouette_score(X, labels)
    scores[n] = score

# Refitting the model to our best found parameter
n_regimes = 3
pipeline.set_params(kmeans__n_clusters=n_regimes)
pipeline.fit(X)

# Plot the scores
fig, ax = plt.subplots()
ax.plot(list(scores.keys()), list(scores.values()), linestyle='--')
ax.scatter(list(scores.keys()), list(scores.values()))
ax.set_title('Silhouette Score for KMeans')
ax.set_xlabel('Number of Clusters')
ax.set_ylabel('Silhouette Score')
ax.grid(True, alpha=0.7, linestyle='--')
plt.show()

From the below plots and metrics, it seems reasonable to believe that our, so far simple, model could be useful in navigating financial markets. Using 2 regimes clustered from our technical indicators, it would indicate that BTC is the best coin to hold 74% of the time but if an investor were to partake in market timing, they may earn more by holding ETH.  

In [ ]:
# Assigning a regimes timeseries
regimes = pd.Series(pipeline.predict(X), data.index)
for coin in coins:

    fig, axs = plt.subplots(1, n_regimes, figsize=(2 * n_regimes, 3))
    fig.suptitle(f'{coin} Returns by Regime')

    # Plot a histogram of each regime with the mean return in the title
    for i in range(n_regimes):

        # Filtering down to the coin's returns within the regime
        regime_returns = data.loc[regimes == i, f'{coin}__Returns']

        # Plotting the histogram
        regime_returns.hist(bins=100, ax=axs[i])
        axs[i].set_title(f'Regime {i}: {regime_returns.mean():.2%}')
        axs[i].grid(True, alpha=0.7, linestyle='--')
        axs[i].set_xlim((-0.15, 0.15))

    plt.tight_layout()

# Print the number of days in each regime
print('\nDays In:')
for i in range(n_regimes):
    print(f'Regime {i}: {regimes.value_counts()[i]}')

# Print the percentage of time in each regime
print('\nPercentage of Time in:')
for i in range(n_regimes):
    print(f'Regime {i}: {regimes.value_counts(normalize=True)[i]:.2%}')

# Print a transition matrix. The probability of transitioning from regime a to b
transition_matrix = pd.crosstab(regimes.shift(), regimes, normalize='index')
print('\nTransition Matrix:')
print(transition_matrix)

In [ ]:

centers = pipeline['kmeans'].cluster_centers_
centers = pd.DataFrame(pipeline['scaler'].inverse_transform(centers), columns=technical_indicators)
centers.index.name = 'Regime'

fig, ax = plt.subplots()
centers.plot(kind='bar', ax=ax)
plt.title('Cluster Centers')
plt.grid(True, alpha=0.7, linestyle='--')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test = train_test_split(X, test_size=0.4, shuffle=False)

# Fit the pipeline to the training set and predict both training and test sets.
pipeline.fit(X_train)
regimes_train = pd.Series(pipeline.predict(X_train), X_train.index)
regimes_test = pd.Series(pipeline.predict(X_test), X_test.index)

# Print the transition matrix of the regime test set
transition_matrix = pd.crosstab(regimes_test.shift(1), regimes_test, normalize='index')
print('\nTest Transition Matrix:')
print(transition_matrix)

In [ ]:
# From this print statement, we see that our strategy is to buy BTC in regime 0 and ETH in regime 1.
portfolios, _ = analyze_regimes_and_get_best_portfolios(returns, regimes_train)

# Create the portfolio timeseries for future testing. It is important to shift 1 day to avoid lookahead bias.
portfolios = create_portfolio_timeseries(regimes_test, portfolios, coins).shift(1)

In [ ]:
# During the testing period, ETH was the best overall coin. So our benchmark to beat is a 100% allocation to ETH during the testing period.
_, benchmark = analyze_regimes_and_get_best_portfolios(returns, regimes_test)
benchmark = create_portfolio_timeseries(regimes_test, benchmark, coins)

# Also make a set of returns for the backtest
returns_test = benchmark.align(returns, join='inner')[1]


In [ ]:
# Plot the portfolio returns and allocation
plot_portfolio_backtest(portfolios, benchmark, returns_test)
plot_portfolio_allocations(portfolios, benchmark)

Reviewing the results of this backtest, our model managed to out perform the benchmark portfolio (The highest returning coin in the testing data). which means that our model may be able to provide investors with insights into coin timing. 

It does appear that our model took less risk than the benchmark portfolio, this is due to the model's ability to buy BTC during regimes with higher volatility. 